# Домашнее задание 4 - Image Captioning
<br>

Задание выполнил(а): *(впишите свои фамилию и имя)*
<br>

**Дедлайн:** 21.12.2020 23:59


### О задании
В этом домашнем задании вы узнаете как построить модель для решения задачи Image Captioning с помощью SAS.
<br>

**Задание построено следующим образом:**
- Задание 1: Загрузка данных в CAS (2 балл)
- Задание 2: Подготовка обучающих данных (3 балла)
- Задание 3: Создание и обучение модели (5 балла)
- Задание 4: Проверка качества модели (5 балл)

In [ ]:
import numpy
import pandas as pd
import os
import csv
import cv2

%matplotlib inline

import swat as sw
from swat import *

from dlpy import *
from dlpy.images import ImageTable
from dlpy.applications import *
from dlpy.image_captioning import *

**Подключение к серверу CAS**<br>
Подключитесь к серверу CAS под своим логином и паролем.<br>
Загрузите следующие Action Sets: 'image' и 'deepLearn'.<br>

In [ ]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = r"/tmp/4fix/trustedcerts.pem"

# Создаем новую сессию CAS:

# Ваш код здесь

# Загружаем action sets:

# Ваш код здесь

**Задание 1: Загрузка данных в CAS**<br>
a) Загрузите изображения в ImageTable. Измените размер изображений на 224х224. Выведите примеры изображений.

In [ ]:
image_table = ImageTable.load_files(s,

In [ ]:
# Измените размер изображений

In [ ]:
# Выведите примеры изображений

б) Загрузите подписи к изображениям с помощью функции create_captions_table

In [ ]:
captions_path = # Путь к файлу с описаниями изображений
delimiter = '\t'
captions = create_captions_table(s,

в) Загрузите веса модели VGG16 (содержатся в архиве с данными) с помощью соответствующего класса в dlpy. 

In [ ]:
VGG_pretrain_file = # Путь к файлу с весами модели VGG16 (прилагается к архиву с обучающими данными)

features_model = VGG16(s,
                       model_table='VGG16',
                       
                       # Use BGR channel means from training
                       # as BGR offset channel values
                       offsets=image_table.channel_means,
                       
                       # Use pretrained .h5 weights file
                       pre_trained_weights=True,
                       
                       # Path to weights file is stored in VGG_pretrain_file
                       pre_trained_weights_file=VGG_pretrain_file,
                       include_top=True,
                       #...
                      )

**Задание 2: Подготовка обучающих данных**<br>
a) Воспользуейтесь функцией get_image_features, чтобы получить фичи для картинок из обучающей выборки с помощью модели VGG16. В качестве целевой перемененной здесь указана переменная '\_filename\_0', содержащяя названия файлов с картинками - это сделано для того, чтобы таблицу с полученными фичами можно было обьеденить с таблицей с подписями используя '\_filename\_0' как id.

In [ ]:
%timeit

dense_layer = 'fc7'
target_var = '_filename_0'

features = get_image_features(s, 

б) С помощью action dlJoin обьедините таблицу с полученными фичами с таблицей с подписями используя '\_filename\_0' как id. Ссылку на полученную таблицу запишите в переменную captions_features (см. https://bit.ly/3pS26TW)

In [ ]:
s.dljoin(

In [ ]:
captions_features = 

в) В исходных данных для каждой картинки дается не один вариант описания (подписи), а сразу 5 различных вариантов. <br>

Каждый из вариантов лежит в отдельном столбце. Для того чтобы обучать модель нам нужно, чтобы все подписи к изображениям лежали в одном столбце - таким образом, на каждом шаге мы будем сравнивать результат модели с одним "верным вариантом". Соответственно нам нужно преобразовать исходную таблицу так, чтобы из 1 строки в исходной таблице получалось 5 строк в результирующей таблице - каждая с одним из пяти вариантов "верного ответа". В исходных данных для каждой картинки дается не один вариант описания (подписи), а сразу 5 различных вариантов. <br>

Помочь нам провести такое преобразование может функция reshape_caption_columns. Изучите документацию к ней и преобразуйте с ее помощью таблицу с данными.

In [ ]:
rnn_input = reshape_caption_columns(s,

Выполните код ниже - он формирует список входных переменных, очевидно, исключая из входных данных колонку с  именами файлов картинок и колонку с предопределенными в исходных данных описаниями.

In [ ]:
columns = list(rnn_input.columns)
columns.remove('caption')
columns.remove('_filename_0')
input_vars = columns

г) Конечно же, модель не сможет работать напрямую с текстовыми описаниями картинок, и для того чтобы ее обучать и использовать, нам потребуется предоставить ей какой-то способ переводить текст в вектор чисел и обратно. Для этого чаще всего используются эмбединнги (векторные представления) слов. Для их обучения требуется много времени и данных, так что мы будем использовать уже обученные эмбединнги - файл с векторными представлениями слов размерностью в 100 измерений уже содержится в архиве данных с домашней работой. Его лишь требуется привести в формат, подходящий для SAS. Дополните код ниже и запустите его, чтобы сделать это.

In [ ]:
raw_embedding_file = # Путь к файлу с эмбеддингами

raw_embedding_dimension = 100

# Use variable 'col_names' to accumulate the 
# generated header strings for all table columns.
col_names = ['term'] + ['_'+str(ii)+'_' for ii in range(1,raw_embedding_dimension+1)]

# Pandas reads in the tab-delimited embedding values
# from the word vector file with no header
df = pd.read_csv(raw_embedding_file, 
                 names=col_names,
                 sep=" ", 
                 index_col=0, 
                 header=None,
                 quoting=csv.QUOTE_NONE)
df.drop(index=df.head(1).index, inplace=True)

# Clean up and omit rows in the table that have 
# reserved or forbidden index character strings.
tmp = [str(df.index[ii]) for ii in range(df.shape[0])]
idx = [ii for ii,txt in enumerate(tmp) if ('"' not in txt) and ("'" not in txt)]
df1 = df.iloc[idx]

pretrained_embedding_file = # Путь к новому файлу, где мы сохраним отформатированные эмбеддинги

df1.to_csv(pretrained_embedding_file, 
           sep='\t', 
           header=True,
           float_format='%5.6f',
           index=True,
           quoting=csv.QUOTE_NONE)
word_embeddings = pretrained_embedding_file

In [ ]:
embeddings = pd.read_csv(pretrained_embedding_file, skipinitialspace=True, index_col=False, delimiter='\t')
s.upload_frame(embeddings, casout=dict(name='word_embeddings', replace=True))

**Задание 3: Создание и обучение модели**<br>
а) В dlpy для моделирования в задаче image captioning применяется отдельный класс под названием ImageCaptioning. Изучите его параметры и создайте с помощью него модель, содержащую 3 блока и 50 нейронов.

In [ ]:
captioning_model = ImageCaptioning(s,

б) Настройте отмеченные ниже параметры обучения и обучите модель. Loss должен опустится ниже 0.2.

In [ ]:
captioning_model.fit(rnn_input,
                     inputs=input_vars,
                     data_specs=[dict(type='numericnominal',
                                      layer='input',
                                      data=input_vars,
                                      numericNominalParms=dict(tokensize=100)
                                     ),
                                 dict(type='text',
                                      layer='output',
                                    data='caption'
                                     )
                                ],
                     # Save best model weights
                     save_best_weights=True,
                     
                     # Word embeddings files
                     text_parms = TextParms(
                         init_input_embeddings='word_embeddings',
                                            model_output_embeddings=dict(name='word_embeddings_out',
                                                                         replace=True
                                                                        )
                                           ),
                     # Random seed key
                     seed=12345,
                     
                     # Use ADAM optimizer
                     optimizer=Optimizer(algorithm=dict(method='ADAM',
                                                        learningRate= # Настройте это параметр
                                                       ),
                                         # Optimization parameters
                                         mini_batch_size= ,# Настройте это параметр
                                         log_level=2,
                                         max_epochs=# Настройте это параметр                                         
                                        ),
                     n_threads=5,
                     train_from_scratch=True
                     )

**Задание 4: Проверка качества модели**<br>
а) Проскорьте полученную модель на обучающей выборке с помощью метода predict. Убедитесь, что вы используете сохраненные веса лучшей модели.

In [ ]:
captioning_model.predict(# Дополните код,
    
                         text_parms=dict(initOutputEmbeddings='word_embeddings_out',
                                         hasOutputTermIds=True,
                                         initInputEmbeddings='word_embeddings',
                                         hasInputTermIds=False,
                                         embeddingTrainable=False
                                        )
                         )

б) Код ниже записывает результаты скоринга в таблицу results и выводит несколько примеров строк из проскоренной таблицы. От вас требуется написать функцию, которая будет выводить n случайных изображений из обучающей выборки, описание из исходных данных и описание, сформированное моделью, для каждой из картинок. Можете использовать **разумное** количество вложенных/дополнительных функций, если у вас есть такая потребность. Не забудьте продемострировать работу функции.

In [ ]:
results = captioning_model.valid_res_tbl
results.head()

In [ ]:
# Напишите код функции здесь

In [ ]:
# Продемонстрируйте работу функции здесь